In [145]:
def read_data_kT(variant=1):
    data_tab = []
    with open('data/kT.txt') as file:
        line = file.readline()
        while line != '':
            line = line.split()
            if variant == 1:
                data_tab.append([int(line[1]), float(line[2])])
            else:
                data_tab.append([int(line[1]), float(line[3])])

            line = file.readline()

    return data_tab
    
variantkT = 1  # 1 or 2
nNewton = 1
data_kT = read_data_kT(variantkT)

# from interpolation.newton import polynom_newton
# from interpolation.work_data import build_short_data_x_from_data
import numpy as np

def k(T, printing=False):
    n = nNewton

    # lndata = ln_data(data_kT)
    # short_data = build_short_data_x_from_data(lndata, x, n + 1)
    # p = polynom_newton(short_data, x, n, printing)
    Tvals = [np.log(data_kT[i][0]) for i in range(len(data_kT))]
    kvals = [np.log(data_kT[i][1]) for i in range(len(data_kT))]
    try:
        res = np.exp(np.interp(np.log(T), Tvals, kvals))
    except Exception as e:
        print(e)
        print(f'T = {T}')
    return res

# print(k(7500))


In [146]:
import plotly.express as px
import pandas as pd

# cnt = 20
# step = (data_kT[-1][0] - data_kT[0][0]) / cnt
step = 500
Tvals = list(map(lambda x: float(x), np.arange(data_kT[0][0], data_kT[-1][0] + step, step )))
kvals = [k(t) for t in Tvals]
tab_Tvals = [data_kT[i][0] for i in range(len(data_kT))]
tab_kvals = [data_kT[i][1] for i in range(len(data_kT))]

df1 = pd.DataFrame({"T": Tvals, "k(T)": kvals, "label": [f"Newton {nNewton}"] * len(Tvals) })
df2 = pd.DataFrame({"T": tab_Tvals, "k(T)": tab_kvals, "label": ["Table"] * len(tab_kvals)})
df = pd.concat([df1, df2])

fig = px.line(df, x="T", y="k(T)", color="label", title="k(T)", markers = True)
fig.show()


In [147]:
c = 3 * 1e10
R = 0.35
Tw = 2000
To = 1e4
p = 4
EPS = 1e-6

def T_r(r):
    return (Tw - To) * ((r / R) ** p) + To


def up_r(r):
    return (3.084 * 1e-4) / (np.exp(4.799 * 1e4 / T_r(r)) - 1)

def du(ri, ui, Fi):
    return -(3 * k(T_r(ri)) * Fi) / c

def dF(ri, ui, Fi):
    if ri == 0 and Fi == 0:
        return 0.5 * k(T_r(ri)) * (up_r(ri) - ui)
    return -Fi / ri + c * k(T_r(ri)) * (up_r(ri) - ui)


In [148]:
# Рунге-Кутта 2

def RungeKutta2_yip1(ri, ui, Fi, h, alpha):
    k1 = du(ri, ui, Fi)
    q1 = dF(ri, ui, Fi)
    k2 = du(ri + h / (2 * alpha), 
              ui + h / (2 * alpha) * k1,
              Fi + h / (2 * alpha) * q1)
    return ui + h * ( (1 - alpha) * k1 + alpha * k2)

def RungeKutta2_zip1(ri, ui, Fi, h, alpha):
    k1 = du(ri, ui, Fi)
    q1 = dF(ri, ui, Fi)
    q2 = dF(ri + h / (2 * alpha), 
              ui + h / (2 * alpha) * k1,
              Fi + h / (2 * alpha) * q1)
    return Fi + h * ( (1 - alpha) * q1 + alpha * q2)

def RungeKutta2(r0, y0, z0, h, alpha, rlast):
    rvals = [r0]
    yvals = [y0]
    Fvals = [z0]

    while rvals[-1] < rlast:
        ri = rvals[-1] + h
        yi = RungeKutta2_yip1(rvals[-1], yvals[-1], Fvals[-1], h, alpha)
        Fi = RungeKutta2_zip1(rvals[-1], yvals[-1], Fvals[-1], h, alpha)
        rvals.append(ri)
        yvals.append(yi)
        Fvals.append(Fi)
        
    return rvals, yvals, Fvals



In [149]:
# Рунге-Кутта 4

def RungeKutta4_yip1(ri, ui, Fi, h):
    k1 = du(ri, ui, Fi)
    q1 = dF(ri, ui, Fi)
    k2 = du(ri + h / 2, ui + h / 2 * k1, Fi + h / 2 * q1)
    q2 = dF(ri + h / 2, ui + h / 2 * k1, Fi + h / 2 * q1)
    k3 = du(ri + h / 2, ui + h / 2 * k2, Fi + h / 2 * q2)
    q3 = dF(ri + h / 2, ui + h / 2 * k2, Fi + h / 2 * q2)
    k4 = du(ri + h, ui + h * k3, Fi + h * q3)
    q4 = dF(ri + h, ui + h * k3, Fi + h * q3)

    return ui + (h / 6) * ( k1 + 2 * k2 + 2 * k3 + k4 )

def RungeKutta4_zip1(ri, ui, Fi, h):
    k1 = du(ri, ui, Fi)
    q1 = dF(ri, ui, Fi)
    k2 = du(ri + h / 2, ui + h / 2 * k1, Fi + h / 2 * q1)
    q2 = dF(ri + h / 2, ui + h / 2 * k1, Fi + h / 2 * q1)
    k3 = du(ri + h / 2, ui + h / 2 * k2, Fi + h / 2 * q2)
    q3 = dF(ri + h / 2, ui + h / 2 * k2, Fi + h / 2 * q2)
    k4 = du(ri + h, ui + h * k3, Fi + h * q3)
    q4 = dF(ri + h, ui + h * k3, Fi + h * q3)

    return Fi + (h / 6) * ( q1 + 2 * q2 + 2 * q3 + q4 )

def RungeKutta4(r0, y0, z0, h, rlast):
    rvals = [r0]
    yvals = [y0]
    Fvals = [z0]

    while rvals[-1] < rlast:
        ri = rvals[-1] + h
        yi = RungeKutta4_yip1(rvals[-1], yvals[-1], Fvals[-1], h)
        Fi = RungeKutta4_zip1(rvals[-1], yvals[-1], Fvals[-1], h)
        rvals.append(ri)
        yvals.append(yi)
        Fvals.append(Fi)
        
    return rvals, yvals, Fvals

In [150]:
h = 1e-2
alpha = 1

r0 = 0
# y0 = khi * up_r(0)
z0 = 0
def func_cond(uR, FR):
    return FR - 0.39 * c * uR

def solution_RungeKutta2():
    def yF_R(khi):
        y0 = khi * up_r(0)
        rvals, yvals, Fvals = RungeKutta2(r0, y0, z0, h, alpha, R)
        return yvals[-1], Fvals[-1]

    # метод половинного деления
    khi_l, khi_r = 1e-10, 1
    yl, Fl = yF_R(khi_l)
    yr, Fr = yF_R(khi_r)
    if not (func_cond(yr, Fr) * func_cond(yl, Fl) < 0):
        print('ERROR')

    while abs(khi_r - khi_l) / khi_l >= EPS:
        khi_mid = (khi_l + khi_r) / 2
        ym, Fm = yF_R(khi_mid)
        if func_cond(ym, Fm) * func_cond(yl, Fl) < 0:
            khi_r = khi_mid
        else:
            khi_l = khi_mid
        
    print(f'khi = {khi_r}')
    khi = khi_r
    y0 = khi * up_r(0)
    # rvals, yvals, Fvals = RungeKutta2(r0, y0, z0, alpha, R)
    return RungeKutta2(r0, y0, z0, h, alpha, R)

def solution_RungeKutta4():
    def yF_R(khi):
        y0 = khi * up_r(0)
        rvals, yvals, Fvals = RungeKutta4(r0, y0, z0, h, R)
        return yvals[-1], Fvals[-1]

    # метод половинного деления
    khi_l, khi_r = 1e-10, 1
    yl, Fl = yF_R(khi_l)
    yr, Fr = yF_R(khi_r)
    if not (func_cond(yr, Fr) * func_cond(yl, Fl) < 0):
        print('ERROR')

    while abs(khi_r - khi_l) / khi_l >= EPS:
        khi_mid = (khi_l + khi_r) / 2
        ym, Fm = yF_R(khi_mid)
        if func_cond(ym, Fm) * func_cond(yl, Fl) < 0:
            khi_r = khi_mid
        else:
            khi_l = khi_mid
        print(f'ksi = {khi_mid}, u = {ym}, F = {Fm}')
        
    print(f'khi = {khi_r}')
    khi = khi_r
    y0 = khi * up_r(0)
    return RungeKutta4(r0, y0, z0, h, R)




In [151]:
def draw_RungeKutta2():
    rvals, yvals, Fvals = solution_RungeKutta2()
    upvals = [up_r(ri) for ri in rvals]
    df1 = pd.DataFrame({"r": rvals, "res": yvals, "label": ["u(r)"] * len(rvals) })
    df2 = pd.DataFrame({"r": rvals, "res": Fvals, "label": ["F(r)"] * len(rvals)})
    df3 = pd.DataFrame({"r": rvals, "res": upvals, "label": ["up(r)"] * len(rvals)})
    # df = pd.concat([df1, df2, df3])

    fig = px.line(df2, x="r", y="res", color="label", title="", markers = True)
    fig.show()

    df = pd.concat([df1, df3])

    fig2 = px.line(df, x="r", y="res", color="label", title="", markers = True)
    fig2.show()


draw_RungeKutta2()

khi = 0.1545075178991855


In [152]:
def draw_RungeKutta4():
    rvals, yvals, Fvals = solution_RungeKutta4()
    upvals = [up_r(ri) for ri in rvals]
    df1 = pd.DataFrame({"r": rvals, "res": yvals, "label": ["u(r)"] * len(rvals) })
    df2 = pd.DataFrame({"r": rvals, "res": Fvals, "label": ["F(r)"] * len(rvals)})
    df3 = pd.DataFrame({"r": rvals, "res": upvals, "label": ["up(r)"] * len(rvals)})
    # df = pd.concat([df1, df2, df3])

    fig = px.line(df2, x="r", y="res", color="label", title="", markers = True)
    fig.show()

    df = pd.concat([df1, df3])

    fig2 = px.line(df, x="r", y="res", color="label", title="", markers = True)
    fig2.show()


draw_RungeKutta4()

ksi = 0.50000000005, u = 1.2350722446469292e-06, F = 1107.3308863532961
ksi = 0.250000000075, u = 5.65739626890199e-07, F = 2909.7772539101675
ksi = 0.1250000000875, u = 2.3107331801183375e-07, F = 3811.0004376886004
ksi = 0.18750000008125, u = 3.9840647245101624e-07, F = 3360.388845799385
ksi = 0.156250000084375, u = 3.1473989523142493e-07, F = 3585.6946417439926
ksi = 0.1406250000859375, u = 2.729066066216295e-07, F = 3698.347539716298
ksi = 0.14843750008515627, u = 2.9382325092652736e-07, F = 3642.0210907301453
ksi = 0.15234375008476564, u = 3.0428157307897604e-07, F = 3613.857866237069
ksi = 0.15429687508457032, u = 3.095107341552006e-07, F = 3599.776253990531
ksi = 0.153320312584668, u = 3.0689615361708854e-07, F = 3606.8170601138
ksi = 0.15380859383461915, u = 3.082034438861443e-07, F = 3603.2966570521658
ksi = 0.15356445320964357, u = 3.075497987516165e-07, F = 3605.056858582984
ksi = 0.15368652352213136, u = 3.0787662131888043e-07, F = 3604.1767578175754
ksi = 0.153747558678375